# 第7章　アドバンスな制御系設計

#### ライブラリの読み込みと関数の定義

ライブラリの読み込みと関数の定義

※以下のpipは初回のみ実施ください。

In [ ]:
%pip install sympy
%pip install control
%pip install numpy scipy control

In [10]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.family'] ='sans-serif' #使用するフォント
plt.rcParams['xtick.direction'] = 'in' #x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams['ytick.direction'] = 'in' #y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams['xtick.major.width'] = 1.0 #x軸主目盛り線の線幅
plt.rcParams['ytick.major.width'] = 1.0 #y軸主目盛り線の線幅
plt.rcParams['font.size'] = 10 #フォントの大きさ
plt.rcParams['axes.linewidth'] = 1.0 # 軸の線幅edge linewidth。囲みの太さ
plt.rcParams['mathtext.default'] = 'regular'
plt.rcParams['axes.xmargin'] = '0' #'.05'
plt.rcParams['axes.ymargin'] = '0.05'
plt.rcParams['savefig.facecolor'] = 'None'
plt.rcParams['savefig.edgecolor'] = 'None'

In [11]:
def linestyle_generator():
    linestyle = ['-', '--', '-.', ':']
    lineID = 0
    while True:
        yield linestyle[lineID]
        lineID = (lineID + 1) % len(linestyle)

# 時間応答のグラフを整える関数
def plot_set(fig_ax, *args):
    fig_ax.set_xlabel(args[0])
    fig_ax.set_ylabel(args[1])
    fig_ax.grid(ls=':')
    if len(args)==3:
        fig_ax.legend(loc=args[2])

# ボード線図を整える関数
def bodeplot_set(fig_ax, *args):
    fig_ax[0].grid(which='both', ls=':')
    fig_ax[0].set_ylabel('Gain [dB]')

    fig_ax[1].grid(which='both', ls=':')
    fig_ax[1].set_xlabel('$\\omega$ [rad/s]')
    fig_ax[1].set_ylabel('Phase [deg]')

    if len(args) > 0:
        fig_ax[1].legend(loc=args[0])
    if len(args) > 1:
        fig_ax[0].legend(loc=args[1])

# 7.1　オブザーバを用いた出力フィードバック制御　p.233

状態観測器（オブザーバ）

PID 制御や位相進み・遅れ補償のような伝達関数モデルをベースとした制御系設計では、制御対象の「出力」をフィードバックしています。

一方、状態空間モデルをベースとする状態フィードバック制御では、制御対象の出力ではなく制御対象の「状態」をフィードバックしており、そして、すべての状態の要素が観測可能であることを前提としています。

しかし現実には、センサなどを使っても、すべての状態が観測できない場合もあります。そのような場合、状態フィードバック制御を行うことができません。そこで役に立つのが、**オブザーバ** です。

これは、図7.1のように、既知の入力 $u$ と出力 $y$ を用いて内部状態 $x$ を推定する動的システムです。

<img src="png/1.png" alt="1" width="300" height="150">

## 7.1.1　同一次元オブザーバ　p.234

同一次元オブザーバ（Full-Order Observer）とは、制御対象（プラント）が持つ状態変数の数（次元）とまったく同じ数の状態変数を持つ状態観測器（オブザーバ）のこと。

よく用いられるオブザーバが、つぎの同一次元オブザーバです。

$$\dot{\hat{\boldsymbol{x}}}(t) = \boldsymbol{A}\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}(y(t) - \boldsymbol{C}\hat{\boldsymbol{x}}(t)) \quad (7.1)$$

ここで、 $\hat{\boldsymbol{x}}$ は推定値です。そして、右辺の第1項、第2項は制御対象のモデルをコピーしたもので、第3項は実際に観測される制御対象の出力 $y$ とオブザーバで推定して計算される $\boldsymbol{C}\hat{\boldsymbol{x}}$ （出力の推定値）の差をフィードバックするものです。

$\boldsymbol{L}$ が設計パラメータで、これをオブザーバゲインといいます。

式 (7.1) に、 $y = \boldsymbol{C}\boldsymbol{x}$ を代入して整理すると、

$$\dot{\hat{\boldsymbol{x}}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t) \quad (7.2)$$

となります。いま、推定誤差を $\boldsymbol{e}(t) = \boldsymbol{x}(t) - \hat{\boldsymbol{x}}(t)$ とすると、

$$\dot{\boldsymbol{e}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\boldsymbol{e}(t) \quad (7.3)$$

となるので、 $\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C}$ が安定であれば $\boldsymbol{e}(t) \to 0$ $(t \to \infty)$ 、つまり、 $\hat{\boldsymbol{x}}(t) \to \boldsymbol{x}(t)$ $(t \to \infty)$ となり、推定値 $\hat{\boldsymbol{x}}$ が制御対象の状態 $\boldsymbol{x}$ に追従します。

オブザーバゲイン $\boldsymbol{L}$ の設計は、状態フィードバックゲイン $\boldsymbol{F}$ の設計のときと同じように極配置で行います。

システムが可観測であれば、任意の位置に極を配置できますので、極を指定して、 $\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C}$ の固有値を指定した極になるように $\boldsymbol{L}$ を決めます。

### ※ 式(7.2)と式(7.3)の導出方法が気になる方は下のセルを展開してください。

### 1. 式(7.1) から 式(7.2) への変形

まず、同一次元オブザーバの式(7.1)を出発点とします。

$$\dot{\hat{\boldsymbol{x}}}(t) = \boldsymbol{A}\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}(y(t) - \boldsymbol{C}\hat{\boldsymbol{x}}(t)) \quad \cdots(7.1)$$

ここで、制御対象の出力方程式 $y(t) = \boldsymbol{C}\boldsymbol{x}(t)$ を代入します。

$$\dot{\hat{\boldsymbol{x}}}(t) = \boldsymbol{A}\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}(\boldsymbol{C}\boldsymbol{x}(t) - \boldsymbol{C}\hat{\boldsymbol{x}}(t))$$

後ろの項を展開します。

$$\dot{\hat{\boldsymbol{x}}}(t) = \boldsymbol{A}\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t) + \boldsymbol{L}\boldsymbol{C}\hat{\boldsymbol{x}}(t)$$

$\hat{\boldsymbol{x}}(t)$ がついている項（第1項と第4項）をまとめます。

$$\dot{\hat{\boldsymbol{x}}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t) \quad \cdots(7.2)$$

これで式(7.2)が導かれました。

### 2. 式(7.3) への変形（推定誤差の微分）

次に、推定誤差 $\boldsymbol{e}(t)$ の時間変化（微分）を考えます。

定義より $\boldsymbol{e}(t) = \boldsymbol{x}(t) - \hat{\boldsymbol{x}}(t)$ ですので、両辺を微分すると以下のようになります。

$$\dot{\boldsymbol{e}}(t) = \dot{\boldsymbol{x}}(t) - \dot{\hat{\boldsymbol{x}}}(t)$$

ここで、右辺の $\dot{\boldsymbol{x}}(t)$ には元のシステムの状態方程式、$\dot{\hat{\boldsymbol{x}}}(t)$ には先ほど導出した式(7.2)を代入します

・$\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t)$

・$\dot{\hat{\boldsymbol{x}}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t)$

これらを代入して引き算を行います。

$$\begin{aligned}
\dot{\boldsymbol{e}}(t) &= \{ \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t) \} - \{ (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t) \} \\
\\
&= \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t) - (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) - \boldsymbol{B}u(t) + \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t)
\end{aligned}$$

入力 $u(t)$ の項（$\boldsymbol{B}u(t)$）はプラスとマイナスで相殺されて消えます。残った項を整理します。

$$\dot{\boldsymbol{e}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{L}\boldsymbol{C}\boldsymbol{x}(t) - (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t)$$

$\boldsymbol{x}(t)$ についてまとめます。

$$\dot{\boldsymbol{e}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\boldsymbol{x}(t) - (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t)$$

共通因数 $(\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})$ でくくります。

$$\dot{\boldsymbol{e}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C}) ( \boldsymbol{x}(t) - \hat{\boldsymbol{x}}(t) )$$

カッコの中身は推定誤差 $\boldsymbol{e}(t)$ そのものです。したがって、

$$\dot{\boldsymbol{e}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\boldsymbol{e}(t) \quad \cdots(7.3)$$

となり、式(7.3)が導かれます。

In [12]:
from control.matlab import ss, acker, lsim
import numpy as np

In [ ]:
# A = '0 1; -4 -5'
# B = '0; 1'
# C = '1 0'
# D = '0'

A = [[0, 1], [-4, -5]]
B = [[0], [1]]
C = [1, 0]
D = [0]
P = ss(A, B, C, D)
print(P)

### オブザーバゲインの設計（極配置）

制御対象を上セルとします。そして、**オブザーバ極**を $-10 \pm 5j$ として、オブザーバゲインを設計します。

設計には、 acker を使いますが、 $(\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})^\top = \boldsymbol{A}^\top + \boldsymbol{C}^\top \boldsymbol{L}^\top$ と考え、 $\boldsymbol{A}^\top$ と $\boldsymbol{C}^\top$ を用いて、 $\boldsymbol{L}^\top$ を計算します。

ただし、任意の位置に極配置するためには、制御対象が可観測である必要があります。

・acker関数を用いてオブザーバゲインを設計するために，A行列とC行列の転置をとったものを用いる

・得られるゲインの転置をとることでオブザーバゲインが得られる

In [ ]:
# オブザーバ極
observer_poles=[-10+5j,-10-5j]

# オブザーバゲインの設計（状態フィードバックの双対）
L = -acker(P.A.T, P.C.T, observer_poles).T
L = L.reshape(-1, 1)
print(L)

# 固有値の確認
print(np.linalg.eigvals(P.A + L @ P.C))

この場合、 $\boldsymbol{L} = [-15 \quad -46]^\top$ となります。

実際、 np.linalg.eigvals(P.A + L @ P.C) を実行すると、 array([-10.+5.j, -10.-5.j]) となりますので、 $\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C}$ の固有値が指定した極になっていることが確認できます。

つぎに、制御対象の初期値を $[-1 \quad 0.5]^\top$ 、オブザーバの初期値を $[0 \quad 0]^\top$ とし、以下のコードを用いてシミュレーションをしてみます。

In [21]:
G = ss(P.A, P.B, np.eye(2), [[0],[0]])
Obs = ss(P.A + L@P.C, np.c_[P.B, -L], np.eye(2), [[0,0],[0,0]] )

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(6, 2.3))

Td = np.arange(0, 3, 0.01)
X0 = [-1, 0.5]
# 入力 u = Fx
u = np.zeros_like(Td)
# u = 10*np.sin(10*T)

x, t, _ = lsim(G, u, Td, X0)
ax[0].plot(t, x[:, 0], ls='-.', label='${x}_1$')
ax[1].plot(t, x[:, 1], ls='-.', label='${x}_2$')

# 出力 y = Cx
y = x[:, 0]
# オブザーバで推定した状態の振る舞い
xhat, t, _ = lsim(Obs, np.c_[u, y], Td, [0, 0])
ax[0].plot(t, xhat[:, 0], label='$\\hat{x}_1$')
ax[1].plot(t, xhat[:, 1], label='$\\hat{x}_2$')

for i in [0, 1]:
    plot_set(ax[i], 't', '', 'best')
    ax[i].set_xlim([0, 3])

ax[0].set_ylim([-2, 2])
ax[1].set_ylim([-3, 2])
ax[0].set_ylabel('$x_1, \\hat{x}_1$')
ax[1].set_ylabel('$x_2, \\hat{x}_2$')

fig.tight_layout()

$$図2　オブザーバによる状態推定$$

上記のコードでは、オブザーバを

$$\dot{\hat{\boldsymbol{x}}}(t) = (\boldsymbol{A} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) + [\boldsymbol{B} \quad -\boldsymbol{L}] \begin{bmatrix} u(t) \\ y(t) \end{bmatrix} \quad (7.4)$$

の形で実装していることに注意してください。

つまり、$u$ と $y=\boldsymbol{C}\boldsymbol{x}$ が入力で $\hat{\boldsymbol{x}}$ が出力となる状態空間モデルが Obs です。

上記のコードの結果は、図 7.2 となります。時間の経過とともに、 $\hat{\boldsymbol{x}}$ が $\boldsymbol{x}$ に追従していることが確認できます。これより、オブザーバにより状態推定が可能であることがわかります。

<img src="png/2.png" alt="1" width="400" height="250">

図 7.3 のように、オブザーバによって推定した状態 $\hat{\boldsymbol{x}}(t)$ を用いて状態フィードバック $u(t) = \boldsymbol{F}\hat{\boldsymbol{x}}(t)$ を施すことができます。このとき、

$$\mathcal{K} : \begin{cases} \dot{\hat{\boldsymbol{x}}}(t) = (\boldsymbol{A} + \boldsymbol{B}\boldsymbol{F} + \boldsymbol{L}\boldsymbol{C})\hat{\boldsymbol{x}}(t) - \boldsymbol{L}y(t) \\ u(t) = \boldsymbol{F}\hat{\boldsymbol{x}}(t) \end{cases} \quad (7.5)$$

が制御器となります。制御対象の出力 $y(t)$ から制御入力 $u(t)$ が計算できます。

また、これをラプラス変換したものは、

$$\mathcal{K}(s) = -\boldsymbol{F}(s\boldsymbol{I} - (\boldsymbol{A} + \boldsymbol{B}\boldsymbol{F} + \boldsymbol{L}\boldsymbol{C}))^{-1}\boldsymbol{L}$$

となります。

### 出力フィードバック制御機の設計

出力フィードバック制御の設計：センサで計測できる情報（出力 $y$）だけを使って、システムを安定化させる制御器を作ること

出力フィードバック制御の結果を図に示します。 

オブザーバと状態フィードバックの併用により、応答が改善されています。

In [33]:
from control.matlab import tf, feedback, initial

# レギュレータ極
regulator_poles = [-5+2j, -5-2j]
# オブザーバ極
observer_poles=[-10+5j,-10-5j]

# 極配置
F = -acker(P.A, P.B, regulator_poles)
# オブザーバゲインの設計（状態フィードバックの双対）
L = -acker(P.A.T, P.C.T, observer_poles).T

F = F.reshape(1, -1)
L = L.reshape(-1, 1)

In [ ]:
K = ss(P.A + P.B @ F + L @ P.C, -L, F, 0)
print('K:\n', K)
print('----------------')
print('K(s)=', tf(K))

# フィードバック系
Gfb = feedback(P, K, sign=1)

In [ ]:
# サイズ3×2.3インチの図を作って、axで軸操作。
fig, ax = plt.subplots(figsize=(3, 2.3))
# 0〜3秒までを0.01秒刻みで定義（時間ベクトル）
Td = np.arange(0, 3, 0.01)

y, t = initial(P, Td, [-1, 0.5]) # 初期状態が [-1, 0.5] のときの初期応答
ax.plot(t,y, ls='-.', label='w/o controller')

y, t = initial(Gfb, Td, [-1, 0.5, 0, 0]) # 推定器付きの制御なら、推定器の状態も含めて初期化
ax.plot(t,y, label='w/ controller')

# ラベル・凡例・グリッドを設定する自作関数か補助関数
plot_set(ax, 't', 'y', 'best')
# 表示範囲を時間0～3秒、出力-1.5～0.5に制限 
ax.set_xlim([0, 3])
ax.set_ylim([-1.5, 0.5])

--- - - 出力フィードバック制御なし

------- 出力フィードバック制御あり

### 7.1.2 定常カルマンフィルタ p.242

定常カルマンフィルタは、雑音のある観測データから、真の状態を最もよく推定する方法の一つ。特に、雑音が確率的に安定（定常）で、システムが時間不変（定数係数）であるときに使う。

オブザーバは制御対象の出力から内部状態を推定するものです。そのため、出力にノイズがのる場合には、正しく推定ができない可能性があります。

そのような場合には、ノイズがすべての周波数帯域でエネルギーが均一な**白色雑音**であると仮定したうえで、推定誤差の二乗平均値を最小化するオブザーバゲイン $L$ を設計します。

**白色雑音**：あらゆる周波数の成分が均等に（同じ強さで）混ざっている雑音のこと。光の「白色光」がすべての色（赤、青、緑など）を均等に含んでいることから、この名前がついている。

ここでは、制御対象

$$\begin{cases}
\dot{x}(t) = \boldsymbol{A}x(t) + \boldsymbol{B}u(t) + v(t) \\
y(t) = \boldsymbol{C}x(t) + w(t)
\end{cases} \tag{7.10}$$

を考えます。ただし、$v$ はシステムノイズ、$w$ は観測ノイズと呼ばれる白色雑音です。さらに、それらの平均と共分散が

$$\begin{aligned}
& \mathbb{E}[v(t)] = 0, \quad \mathbb{E}[v(t)v(\tau)^{\top}] = \boldsymbol{Q}\delta(t-\tau) \\
& \mathbb{E}[w(t)] = 0, \quad \mathbb{E}[w(t)w(\tau)^{\top}] = \boldsymbol{R}\delta(t-\tau)
\end{aligned}$$

であり、$v$ と $w$ は互いに独立であるとします$^{*4}$。このとき、つぎの結果が知られており、この方法で設計したオブザーバは、定常カルマンフィルタとも呼ばれます。

----------------------------------------------------------------------------------------------------------------------------------------------------

$\boldsymbol{Q} > 0, \boldsymbol{R} > 0$ に対して、推定誤差の二乗平均値

$$J = \mathbb{E} \left[ (x(t) - \hat{x}(t))^{\top} (x(t) - \hat{x}(t)) \right] \tag{7.11}$$

を最小化するオブザーバ（定常カルマンフィルタ）は、

$$\dot{\hat{x}}(t) = \boldsymbol{A}\hat{x}(t) + \boldsymbol{B}u(t) - \boldsymbol{L}_{\mathrm{opt}} (y(t) - \boldsymbol{C}\hat{x}(t)) \tag{7.12}$$

であり、$\boldsymbol{L}_{\mathrm{opt}}$ の値は

$$\boldsymbol{L}_{\mathrm{opt}} = -\boldsymbol{P}\boldsymbol{C}^{\top}\boldsymbol{R}^{-1} \tag{7.13}$$

となる。ただし、$\boldsymbol{P} = \boldsymbol{P}^{\top} > 0$ は、リカッチ方程式

$$\boldsymbol{A}\boldsymbol{P} + \boldsymbol{P}\boldsymbol{A}^{\top} + \boldsymbol{Q} - \boldsymbol{P}\boldsymbol{C}^{\top}\boldsymbol{R}^{-1}\boldsymbol{C}\boldsymbol{P} = 0 \tag{7.14}$$

を満たす唯一の正定対称解である。

----------------------------------------------------------------------------------------------------------------------------------------------------

定常カルマンフィルタは、lqe 関数を用いて設計することができます。

---------------------------------------------------------

L, P, E = lqe(A, Bv, C, QN, RN)

L = -L

----------------------------------------------------------

ここで、引数の A, Bv, C は、システム $\dot{x} = \boldsymbol{A}x + \boldsymbol{B}u + \boldsymbol{B}_v v$, $y = \boldsymbol{C}x + w$ の $\boldsymbol{A}$ 行列、$\boldsymbol{B}_v$ 行列、$\boldsymbol{C}$ 行列です。また、QN と RN はノイズの共分散です。

それから、返り値の L はオブザーバゲイン、P はリカッチ方程式の解、E は閉ループ系の極（$\boldsymbol{A} - \boldsymbol{L}\boldsymbol{C}$ の固有値）です。

式(7.12)の $\boldsymbol{L}_{\mathrm{opt}}$ は、lqe で求まる L に対して、 L = -L とすることで得られます。以下では、

$$\begin{cases}
\dot{x}(t) = \begin{bmatrix} 0 & 1 \\ -4 & -5 \end{bmatrix} x(t) + \begin{bmatrix} 0 \\ 1 \end{bmatrix} (u(t) + v(t)) \\
y(t) = \begin{bmatrix} 1 & 0 \end{bmatrix} x(t) + w(t)
\end{cases} \tag{7.15}$$

に対して、定常カルマンフィルタを設計します。なお、システムノイズと観測ノイズの共分散はどちらも1とします。

以下のコードを実行すると、図が得られます。上の図がカルマンフィルタに入力されるノイズを含んだ制御入力と出力です。これらから推定した状態が下の図です。波線が真の状態です。

この結果をみると、ノイズがある場合でもカルマンフィルタにより状態が推定できることがわかります。

In [38]:
from control.matlab import ss, lqe, lsim

# ss: 状態空間表現（State Space）を作成する関数
#     形式: ss(A, B, C, D)
#     例: システム行列 A, B, C, D を使って状態空間モデルを定義できる。

# lqe: カルマンフィルタ（最適状態推定器）の設計を行う関数（Linear Quadratic Estimator）
#      形式: lqe(A, G, C, Q, R)
#      出力: L（カルマンゲイン）, P（推定誤差共分散）, E（オブザーバの極）

# lsim: 入力信号に対して線形システムの応答（出力と状態）をシミュレーションする関数
#       形式: lsim(sys, U, T, X0)
#       sys: 状態空間モデル（ssで定義）
#       U: 入力信号（行列 or 配列）
#       T: 時間ベクトル
#       X0: 初期状態
#       出力: 応答（出力 y）、時間ベクトル t、状態 x

In [ ]:
QN = 1  # プロセスノイズ（状態雑音）の共分散 QN（スカラー）。モデル誤差の大きさを表す。
RN = 1  # 観測ノイズの共分散 RN（スカラー）。センサの誤差の大きさを表す。

# 定常カルマンフィルタ（LQE: Linear Quadratic Estimator）によって観測ゲイン L を設計
# P.A: システム行列 A
# P.B: 入力行列（ここではプロセスノイズの駆動行列 G として使用）
# P.C: 出力行列 C
# QN: プロセスノイズの共分散
# RN: 観測ノイズの共分散
# lqe は (A, G, C, Q, R) に対して最適なオブザーバゲイン L を計算する
L, _, _ = lqe(P.A, P.B, P.C, QN, RN)

# 計算されたゲイン L を符号反転（制御理論では L = -K の形で使うことが多いため）
L = -L

# 計算された観測ゲイン L を表示
print(L)

In [ ]:
# 時間軸 Td の設定（0 から 6 秒まで、0.01 秒刻み）
Td = np.arange(0, 6, 0.01)

# 入力信号 u：2つの異なる周波数を混ぜた正弦波
u = 0.5 * np.sin(6*Td) + 0.5 * np.cos(8*Td)

# プロセスノイズ QN と 観測ノイズ RN の分散
QN, RN = 1, 1

# プロセスノイズ w（標準正規分布に従うノイズ、分散 QN）
w = np.random.normal(loc=0, scale=np.sqrt(QN), size=len(Td))

# 観測ノイズ v（標準正規分布に従うノイズ、分散 RN）
v = np.random.normal(loc=0, scale=np.sqrt(RN), size=len(Td))

# w の分散と平均を確認（期待値とばらつきをチェック）
print(np.var(w))
print(np.mean(w))

# カルマンゲイン L の設計（lqe = Linear Quadratic Estimator）
L, _, _ = lqe(P.A, P.B, P.C, QN, RN)

# L を符号反転（制御でよく用いる形式）
L = -L
print(L)

# オブザーバモデルの構築：Lを用いたシステム（状態推定用）
Obs = ss(P.A + L@P.C, np.block([P.B, -L]), np.eye(2), np.zeros([2,2]))

# 初期状態
X0 = [-0.3, 0.2]

# ノイズ付き入力 u+w を使って真の状態 x をシミュレーション
_, t, x = lsim(P, u+w, Td, X0)

# 理想状態（雑音なし）xorg をシミュレーション（比較用）
_, t, xorg = lsim(P, u, Td, X0)

# 観測値 y = Cx + v（出力に観測ノイズを加える）
y = x[:, 0]+v

# グラフ描画の準備：2行2列のサブプロット
fig, ax = plt.subplots(2,2, figsize=(6, 4.6))

# 出力 y を描画（雑音含む）
ax[0,1].plot(t, y, ls='-.', label='$y$', c='gray', lw=0.5)

# 真の状態 x1, x2（雑音なし）を描画
ax[1,0].plot(t, xorg[:,0], ls='--', label='$x_1$', c='k')
ax[1,1].plot(t, xorg[:,1], ls='--', label='$x_2$', c='k')

# 入力 u+w（ノイズ付き）を描画
ax[0,0].plot(t, u+w, ls='-.', label='$u+w$', c='gray', lw=0.5)

# 状態オブザーバを使って状態 x を推定（xhat）
xhat, t, _ = lsim(Obs, np.c_[u, y], Td, [0, 0])

# 推定結果（オブザーバ出力）を描画
ax[1,0].plot(t, xhat[:, 0], label='$\\hat{x}_1$', c='k')
ax[1,1].plot(t, xhat[:, 1], label='$\\hat{x}_2$', c='k')

# 理想入力 u を描画（比較用）
ax[0,0].plot(t, u, label='$u$', c='k', ls='--')

# 各サブプロットの共通設定
for i in [0, 1]:
    for j in [0, 1]:
        plot_set(ax[i,j], '$t$', '', 'best')
        ax[i,j].set_xlim([0, 6])

# y 軸ごとの範囲とラベル設定
ax[0,1].set_ylim([-4, 4])
ax[0,1].set_ylabel('$y$')

ax[0,0].set_ylabel('$u$')
ax[0,0].set_ylim([-4, 4])

ax[1,0].set_ylim([-0.4, 0.4])
ax[1,0].set_ylabel('$x_1$')

ax[1,1].set_ylim([-0.4, 0.4])
ax[1,1].set_ylabel('$x_2$')

# 図全体のレイアウト調整
fig.tight_layout()
